In [ ]:
# !git clone https://github.com/facebookresearch/segment-anything-2

In [1]:
%cd segment-anything-2

/workspaces/sam2_video_analysis/notebooks/segment-anything-2


In [2]:
!pip install -e .

Obtaining file:///workspaces/sam2_video_analysis/notebooks/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Building editable for SAM-2 (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Building editable for SAM-2 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [127 lines of output]
      /tmp/pip-build-env-y1kvkmc8/overlay/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
        cpu = _conversion_method_template(device=torch.device("cpu"))
      running editable_wheel
      creating /tmp/pip-wheel-8ophtd0f/.tmp-7r3u25oy/SAM_2.egg-info


In [3]:
!pip install supervision jupyter_bbox_widget autodistill autodistill-grounded-sam-2


  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: -11
  ╰─> [0 lines of output]
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!mkdir -p checkpoints
%cd checkpoints
!wget https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt
!wget https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt
!wget https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt
!wget https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt
%cd ..


In [ ]:
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from PIL import Image
import numpy as np
import supervision as sv

# Load model
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINT = "checkpoints/sam2_hiera_large.pt"
CONFIG = "sam2_hiera_l.yaml"

sam2_model = build_sam2(CONFIG, CHECKPOINT, device=DEVICE, apply_postprocessing=False)

# Load image
image_path = 'path/to/your/image.jpg'
image = np.array(Image.open(image_path).convert("RGB"))

# Generate masks
mask_generator = SAM2AutomaticMaskGenerator(sam2_model)
masks = mask_generator.generate(image)

# Visualize masks
mask_annotator = sv.MaskAnnotator()
detections = sv.Detections.from_sam(masks)
detections.class_id = [i for i in range(len(detections))]
annotated_image = mask_annotator.annotate(image, detections)

sv.plot_image(image=annotated_image, size=(8, 8))


In [ ]:
from autodistill_grounded_sam_2 import GroundedSAM2
from autodistill.detection import CaptionOntology
from autodistill.utils import plot
import cv2
import supervision as sv

# Define an ontology to map class names to Grounded SAM 2 prompts
base_model = GroundedSAM2(
    ontology=CaptionOntology(
        {
            "basketball": "basketball",
            "player": "player",
            "court line": "court line",
            "hoop": "hoop"
        }
    )
)

# Run inference on an image
image_path = 'path/to/your/image.jpg'
results = base_model.predict(image_path)

image = cv2.imread(image_path)
mask_annotator = sv.MaskAnnotator()
annotated_image = mask_annotator.annotate(image.copy(), detections=results)

sv.plot_image(image=annotated_image, size=(8, 8))
